In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi


In [ ]:
!pip install -q ultralytics

/kaggle/input/fracatlas-dataset/Fracatlas/validation/lables

/kaggle/input/fracatlas-dataset/Fracatlas/test/labels

/kaggle/input/fracatlas-dataset/Fracatlas/train/labels

there is different spelling for "labels" and "lables"

In [ ]:
!cp -r /kaggle/input/fracatlas-dataset/Fracatlas /kaggle/working/Fracatlas


In [ ]:
!mv /kaggle/working/Fracatlas/validation/lables /kaggle/working/Fracatlas/validation/labels


In [ ]:
import os

for split in ["train", "validation", "test"]:
    print(split)
    print("images:", len(os.listdir(f"/kaggle/working/Fracatlas/{split}/images")))
    print("labels:", len(os.listdir(f"/kaggle/working/Fracatlas/{split}/labels")))


In [ ]:
import cv2, random, os
import matplotlib.pyplot as plt

base_path = "/kaggle/working/Fracatlas"
img_dir = f"{base_path}/train/images"
lbl_dir = f"{base_path}/train/labels"

def show_random_image():
    img_name = random.choice(os.listdir(img_dir))
    img_path = f"{img_dir}/{img_name}"
    lbl_path = f"{lbl_dir}/{img_name.replace('.jpg','.txt')}"

    img = cv2.imread(img_path)
    h, w, _ = img.shape

    with open(lbl_path) as f:
        for line in f:
            cls, xc, yc, bw, bh = map(float, line.split())
            x1 = int((xc - bw/2) * w)
            y1 = int((yc - bh/2) * h)
            x2 = int((xc + bw/2) * w)
            y2 = int((yc + bh/2) * h)
            cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)

    plt.figure(figsize=(5,5))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

for _ in range(5):
    show_random_image()


In [ ]:
yaml_text = f"""
path: {base_path}
train: train/images
val: validation/images
test: test/images

names:
  0: fracture
"""

with open("/kaggle/working/fracatlas_fixed.yaml", "w") as f:
    f.write(yaml_text)

print("Dataset YAML ready")


BASELINE: YOLOv8n @ 640

In [ ]:
from ultralytics import YOLO

model_n_640 = YOLO("yolov8n.pt")

model_n_640.train(
    data="/kaggle/working/fracatlas_fixed.yaml",
    epochs=60,
    imgsz=640,
    batch=16,
    project="FracAtlas_Project",
    name="yolov8n_640"
)


YOLOv8s @ 640

In [ ]:
model_s_640 = YOLO("yolov8s.pt")

model_s_640.train(
    data="/kaggle/working/fracatlas_fixed.yaml",
    epochs=60,
    imgsz=640,
    batch=16,
    project="FracAtlas_Project",
    name="yolov8s_640"
)


In [ ]:
!cat /kaggle/working/fracatlas_fixed.yaml


YOLOv8n @ 960

In [ ]:
model_n_960 = YOLO("yolov8n.pt")

model_n_960.train(
    data="/kaggle/working/fracatlas_fixed.yaml",
    epochs=60,
    imgsz=960,
    batch=8,
    project="FracAtlas_Project",
    name="yolov8n_960"
)


LOAD MODELS

In [ ]:
!find /kaggle/working -name best.pt


In [ ]:
from ultralytics import YOLO

model_n_640 = YOLO("/kaggle/working/runs/detect/FracAtlas_Project/yolov8n_960/weights/best.pt")
model_s_640 = YOLO("/kaggle/working/runs/detect/FracAtlas_Project/yolov8s_640/weights/best.pt")
model_n_960 = YOLO("/kaggle/working/runs/detect/FracAtlas_Project/yolov8n_640/weights/best.pt")


In [ ]:
metrics_n_640 = model_n_640.val(data="/kaggle/working/fracatlas_fixed.yaml", split="test")
metrics_s_640 = model_s_640.val(data="/kaggle/working/fracatlas_fixed.yaml", split="test")
metrics_n_960 = model_n_960.val(data="/kaggle/working/fracatlas_fixed.yaml", split="test")

print("YOLOv8n@640:", metrics_n_640)
print("YOLOv8s@640:", metrics_s_640)
print("YOLOv8n@960:", metrics_n_960)


## Model Comparison Results

| Model | Image Size | Precision | Recall | mAP@0.5 | mAP@0.5:0.95 |
|--------|-----------|-----------|--------|--------|--------------|
| YOLOv8n | 640 | 0.659 | 0.328 | 0.406 | 0.163 |
| YOLOv8s | 640 | **0.817** | 0.403 | **0.490** | 0.187 |
| YOLOv8n | 960 | 0.755 | **0.413** | 0.484 | **0.202** |

### Key Observations
- Increasing model capacity (YOLOv8s) improves precision and overall detection accuracy.
- Increasing image resolution improves recall and fine-grained localization of fractures.
- Best recall is achieved using higher resolution input, indicating that small fracture regions benefit from increased spatial detail.


In [ ]:
model_n_640.predict(source=f"{base_path}/test/images", save=True, conf=0.25)
model_s_640.predict(source=f"{base_path}/test/images", save=True, conf=0.25)
model_n_960.predict(source=f"{base_path}/test/images", save=True, conf=0.25)


## Discussion

The baseline YOLOv8n model shows limited recall, indicating difficulty in detecting small and low-contrast fracture regions. 
This is expected because fracture lines are often thin and occupy very small areas in the image.

Using a larger model (YOLOv8s) improves precision and mAP, suggesting that higher model capacity helps in learning complex fracture patterns.

Using higher resolution input improves recall and localization accuracy, supporting the hypothesis that fracture detection is a small-object detection problem.

Overall, both architectural capacity and input resolution contribute to performance, but resolution plays a stronger role in improving recall, which is crucial in medical applications.


ERROR ANALYSIS

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import random

gt_label_dir = "/kaggle/working/Fracatlas/test/labels"
pred_label_dir = "/kaggle/working/runs/detect/predict3"
img_dir = "/kaggle/working/Fracatlas/test/images"


In [ ]:
missed_images = []

for file in os.listdir(gt_label_dir):
    pred_path = os.path.join(pred_label_dir, file)
    
    if not os.path.exists(pred_path) or os.path.getsize(pred_path) == 0:
        missed_images.append(file.replace(".txt", ".jpg"))

print("Number of missed fracture images:", len(missed_images))
missed_images[:5]


MISSED FRACTURES

In [ ]:
def show_missed():
    img_name = random.choice(missed_images)
    img = cv2.imread(os.path.join(img_dir, img_name))
    
    plt.figure(figsize=(5,5))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Missed Fracture (False Negative)")
    plt.axis("off")
    plt.show()

for _ in range(5):
    show_missed()


IMAGES WITH DETECTIONS

In [ ]:
detected_images = []

for file in os.listdir(pred_label_dir):
    if os.path.getsize(os.path.join(pred_label_dir, file)) > 0:
        detected_images.append(file.replace(".txt", ".jpg"))

print("Images with detections:", len(detected_images))
detected_images[:5]


SOME DETECTED CASES

In [ ]:
def show_detected():
    img_name = random.choice(detected_images)
    img = cv2.imread(os.path.join(img_dir, img_name))
    
    plt.figure(figsize=(5,5))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Detected Fracture")
    plt.axis("off")
    plt.show()

for _ in range(5):
    show_detected()


## Error Analysis (Qualitative)

Missed detections mainly occur in images where fractures are:
- very thin (hairline cracks)
- low contrast relative to surrounding bone
- partially occluded by overlapping structures

Correct detections are mostly observed when:
- fracture region has strong contrast
- fracture covers relatively larger area

This indicates that fracture detection is a small-object and low-contrast problem, which explains why higher resolution input improves recall.
